In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import plotly
from pyathena import connect
import pandas as pd

In [7]:
q="""
select
 t1.date,
 t1.pageCount,
 t2.fileCount
from
(
SELECT
 count(*) pageCount, 
 g.date
 FROM 
 ems.google_analytics g cross join unnest (g.hits) as t(a)
where 
 g.date >= '20210601' and g.date <= '20210630'
 and a.page.hostname like '%asf%'
 and a.eventinfo is not null and a.eventinfo.eventcategory='general download - master'
group by 
 g.date
 ) t1,
( 
 select
  sum(distdatarowid) fileCount,
  replace(cast(d.requestDate as varchar(8)), '-', '') distDate
 from  ems.distdailysummary d
 where 
  d.requestDate between timestamp '2021-06-01' and timestamp '2021-06-30'
  and d.provider='ASF'
 group by requestDate ) t2
where
 t1.date = t2.distDate
order by
 t1.date
"""
conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')
pd.options.display.float_format = '{:,.0f}'.format
df = pd.read_sql(q, conn)
df

,date,pageCount,fileCount
0,20210601,7095,135533
1,20210602,6684,190284
2,20210603,7535,174479
3,20210604,5382,153951
4,20210605,2358,87320
5,20210606,3623,67800
6,20210607,5575,148132
7,20210608,5125,88169
8,20210609,5237,185312
9,20210610,5500,150646


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['date'], y=df['pageCount'], name="page event count"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['date'], y=df['fileCount'], name="file count"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="ASF, June , Web Download Events v File Distribution"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Web Download Events.</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Distribution # of files.</b>", secondary_y=True)

fig.show()
fig.write_html("timeseries_" + "test" + ".html")

In [5]:
# Web/Dist Matchup by Geolocation

create table analyst_1.webDistMatchUp as 
select
 length(eventInfo.eventLabel) LabelLength,
 substr(eventInfo.eventLabel, length(eventInfo.eventLabel) - 12) endOfLabel, 
 st_asText(webPoint) webPoint_Text,
 st_distance(webPoint, distPoint) wd_distance,
 requestDateTime,
 g.date,
 g.hour,
 g.minute,
 product,
 provider,
 orgname,
 eventinfo.eventlabel,
 requestPath,
 visitNumber,
 hitNumber,
 pagepath
from 
  (select 
    st_point(cast(g.geonetwork.longitude as double), cast(g.geonetwork.latitude as double) ) webPoint,
    g.date, h.hour, h.minute,
    g.visitNumber,
    h.page.pagepath,
    h.eventinfo,
    h.hitNumber
  from
  ems.google_analytics g cross join unnest(g.hits) as t(h)
  where g.date = '20210401'  
   and h.page.hostname <> '127.0.0.1'
   and h.page.hostname <> 'localhost'
   and h.eventinfo is not null
   and h.eventinfo.eventcategory='general download - master'
) g,
  (select 
    st_point(cast(longitude as double), cast(latitude as double)) distPoint,
    provider,
    product,
   orgname,
   requestPath,
   requestDateTime
   from ems.distdata 
   where requestDate = cast('2021-04-01' as date)
       and longitude <> '' and latitude <> ''
                    ) d  
where
 st_distance(webPoint, distPoint) <= 0.005
 and 
  date_parse(g.date || ' ' || cast(g.hour as varchar) || ':' || cast(g.minute as varchar), '%Y%m%d %H:%i') between requestDateTime - interval '1' hour and requestDateTime + interval '1' hour
  and requestPath like '%' || substr(eventInfo.eventLabel, length(eventInfo.eventLabel) - 12) || '%'
limit 100